In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 878.7/878.7 kB 14.5 MB/s eta 0:00:00


In [2]:
!pip install -q supervision roboflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.1 MB/s eta 0:00:00


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="6DPqoeKxBWbUQuPOz9wS")
project = rf.workspace("learning-czw9n").project("bottle-0x7qy")
version = project.version(3)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to bottle-3 in yolov8:: 100%|██████████| 1113/1113 [00:00<00:00, 7775.76it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="6DPqoeKxBWbUQuPOz9wS")
project = rf.workspace("learning-czw9n").project("bottle-0x7qy")
version = project.version(3)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
!yolo task=detect \
mode=train \
model=yolov8s.pt \
data=/content/bottle-3/data.yaml \
epochs=25 \
imgsz=350

100% 21.5M/21.5M [00:00<00:00, 185MB/s]
Ultralytics 8.3.24 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/bottle-3/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=350, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, sho

In [5]:
# Add this after training
from google.colab import drive
drive.mount('/content/drive')  # Mount Google Drive
!cp /content/runs/detect/train/weights/best.pt /content/drive/MyDrive/latest_best.pt  # Adjust paths as needed


Mounted at /content/drive
cp: cannot stat '/content/runs/detect/train/weights/best.pt': No such file or directory


In [6]:
from google.colab import drive
import os

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Create a folder for captured frames
folder_name = 'captured_frames'
drive_folder_path = f'/content/drive/MyDrive/{folder_name}'

# Create the directory if it doesn't exist
if not os.path.exists(drive_folder_path):
    os.makedirs(drive_folder_path)
    print(f"Created folder: {drive_folder_path}")
else:
    print(f"Folder already exists: {drive_folder_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folder already exists: /content/drive/MyDrive/captured_frames


In [9]:
import time
from roboflow import Roboflow
from google.colab import drive
import os
import shutil

def clear_captured_frames(folder_path):
    """Delete all files from the specified folder."""
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Delete the file
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Delete the directory
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

# Mount Google Drive
drive.mount('/content/drive')

# Define paths for frames and model
frames_path = '/content/drive/MyDrive/captured_frames'
data_path = '/content/bottle-3/data.yaml'  # Your existing dataset YAML file
model_path = '/content/drive/MyDrive/model_checkpoints/best.pt'  # Path for the model checkpoint

while True:
    # Check if new frames are available
    if os.listdir(frames_path):  # If new frames are present
        shutil.copytree('/content/drive/MyDrive/captured_frames', '/content/bottle-3/train/images',dirs_exist_ok=True)
        clear_captured_frames('/content/drive/MyDrive/captured_frames')
        print("New frames detected, retraining model...")

        # Optionally: Move or process frames for training
        # Code to integrate frames into the dataset (e.g., Roboflow or YOLO format)

        # Retrain model with updated data
        !yolo task=detect mode=train model=yolov8s.pt data={data_path} epochs=5 imgsz=350

        # After training, save the updated model to Google Drive
        # Save logic here (it might be already handled by YOLO)

    else:
        print("No new frames to add for retraining.")

    # Wait before the next check
    time.sleep(60)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
New frames detected, retraining model...
Ultralytics 8.3.25 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/bottle-3/data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=350, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, 

KeyboardInterrupt: 